In [32]:
import gurobipy as gp
from gurobipy import GRB
import sys
import numpy as np
import datetime
import copy

In [33]:
NITER = 2000

# Acá se definen los datos
m = 50
n = 2000
M = range(m)
N = range(n)

B = 1
Tk = 10000
alpha = 0.99
accept = 0
probT = 0


np.random.seed(71096)

c = np.random.randint(3,7,size=(m,n))
f = np.random.randint(10,30,size=m)

In [34]:
# Se define el problema completo original

modelf = gp.Model('Location and Asignment Full Model')
modelf.setParam('OutputFlag', True) # turns on solver chatter

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


In [35]:
# Esta es la definición de variables y restricciones del problema completo original

xf = modelf.addVars(M,N,vtype=GRB.BINARY)
yf = modelf.addVars(M,vtype=GRB.BINARY)

rest_demand = modelf.addConstrs(((sum(xf[i,j] for i in M) == 1) for j in N),name = "DEMANDF")
rest_assign = modelf.addConstrs(((sum(xf[i,j] for j in N) <= n*yf[i]) for i in M),name = "ASSIGN")


modelf.setObjective((sum(sum(c[i,j]*xf[i,j] for i in M) for j in N)+sum(f[i]*yf[i] for i in M)),GRB.MINIMIZE)

In [36]:
# Aquí se declara el problema relajado lineal y se resuelve
modelr = modelf.relax()
modelr.optimize()

valor_relajado = modelr.objVal

print( '\n Objectivo modelo relajado =', valor_relajado,'\n')


Derived model does not contain these changes.
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00

 Objectivo modelo relajado = 0.0 



In [37]:
# Acá se define el modelo restringido a las x

model = gp.Model('Location and Asignment modelo restringido a x')
model.setParam('OutputFlag', False) # turns off solver chatter

y = [0.0]*m
j0 = np.random.randint(0,m-1)
y[j0] = 1

costo_y = sum(f[i]*y[i] for i in M)

x = model.addVars(m,n,vtype=GRB.BINARY)

rest_rel = model.addConstrs(((sum(x[i,j] for j in N) <= n*y[i]) for i in M),name = "ASSIGN_REL")
rest_assign = model.addConstrs(((sum(x[i,j] for i in M) == 1) for j in N),name = "DEMAND_REL")

model.setObjective((sum(sum(c[i,j]*x[i,j] for i in M) for j in N)),GRB.MINIMIZE)
 
model.update()
model.optimize()

valor = costo_y + model.objVal
best = valor

In [38]:
# Este es el ciclo principal del Simmulated Annealing
begin_time = datetime.datetime.now()



for k in range(1, NITER):

#   Ahora evaluamos una nueva solución vecina. La vecindad se define por aperturas
#   y cierres individuales de localizaciones en forma aleatoria

    yvecino = copy.deepcopy(y)
    j0 = np.random.randint(0,m-1)
    yvecino[j0] = 1-yvecino[j0]
    
    # Tenemos el vecino, ahora evaluamos el valor
    # Hay que coreer nuevamente el problema para evaluar.
    rest_rel[j0].RHS = n*yvecino[j0]
    
    model.update()
    model.optimize()
    
    costo_vecino = sum(f[i]*yvecino[i] for i in M)
    valor_vecino = costo_vecino + model.objVal

#   Se despliega el valor de la solución actual.
    
        
    # Ahora se decide si aceptar o no la solución vecina.
    
    valor_old = valor
    if valor_vecino <= valor:
        y = copy.deepcopy(yvecino)
        valor = valor_vecino
        accept = 0
    else:
        probT = np.exp(-(valor_vecino - valor)/(B*Tk))
        prob = np.random.rand()
        if probT > prob:
            y = copy.deepcopy(yvecino)
            valor = valor_vecino
            accept = 1
        else:
            # se revierte a la solución inicial de la iteración en el modelo
            rest_rel[j0].RHS = n*y[j0]
            model.update()
            accept = 0
        
    if valor < best:
        best = valor
        
    # se actualiza la temperatura
    print("%8.0f %8.4f %8.4f %10.4f  %10.4f %4.0f" % (k, valor_old, valor_vecino, Tk, probT, accept))
    Tk = alpha*Tk
    
print('\n mejor valor encontrado: ',best)
print(f'Tiempo de ejecución: {datetime.datetime.now() - begin_time}')


       1 8996.0000 7810.0000 10000.0000      0.0000    0
       2 7810.0000 7191.0000  9900.0000      0.0000    0
       3 7191.0000 6834.0000  9801.0000      0.0000    0
       4 6834.0000 6641.0000  9702.9900      0.0000    0
       5 6641.0000 6522.0000  9605.9601      0.0000    0
       6 6522.0000 6445.0000  9509.9005      0.0000    0
       7 6445.0000 6520.0000  9414.8015      0.9921    1
       8 6520.0000 6441.0000  9320.6535      0.9921    0
       9 6441.0000 6391.0000  9227.4469      0.9921    0
      10 6391.0000 6358.0000  9135.1725      0.9921    0
      11 6358.0000 6340.0000  9043.8208      0.9921    0
      12 6340.0000 6334.0000  8953.3825      0.9921    0
      13 6334.0000 6337.0000  8863.8487      0.9997    1
      14 6337.0000 6342.0000  8775.2102      0.9994    1
      15 6342.0000 6342.0000  8687.4581      0.9994    0
      16 6342.0000 6331.0000  8600.5835      0.9994    0
      17 6331.0000 6338.0000  8514.5777      0.9992    1
      18 6338.0000 6356.0000  8